## TRANSFORMERS DATA CREATION

#### DATA CLEANING

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
input_csv = pd.read_csv('/Volumes/T7/ML/Data/Transformers/English_to_hindi_translation.csv')

In [ ]:
input_csv.head()

In [ ]:
cleaned=input_csv.drop(['source'], axis= 1)

In [ ]:
cleaned.head()

In [ ]:
cleaned = cleaned.dropna()

In [ ]:
cleaned.isna().sum()

In [ ]:
cleaned.describe()

In [ ]:
unk_token = '<unk>'

In [ ]:
import re

def remove_punctuations_and_lower(sentence):
    clean_sentence = re.sub(r'[^\w\s]','', sentence)
    
    # \w -> non word \s -> non space, so, \w\s replaces any special characters other than a non word or a non space.
    return clean_sentence.lower()

In [ ]:
cleaned['english_sentence_no_punctuations'] = cleaned['english_sentence'].apply(remove_punctuations_and_lower)
cleaned['hindi_sentence_no_punctuations'] = cleaned['hindi_sentence'].apply(remove_punctuations_and_lower)

In [ ]:
cleaned.head()

In [ ]:
from string import digits
remove_digits = str.maketrans('', '', digits) #digits to be removed

def remove_digits_from_english(sentence): 
     return sentence.translate(remove_digits)
    
def remove_digits_from_hindi(sentence): 
     return re.sub("[२३०८१५७९४६]", "", sentence)

In [ ]:
cleaned['english_cleaned'] = cleaned['english_sentence_no_punctuations'].apply(remove_digits_from_english)
cleaned['hindi_cleaned'] = cleaned['hindi_sentence_no_punctuations'].apply(remove_digits_from_hindi)

In [ ]:
cleaned = cleaned.drop(['english_sentence_no_punctuations','hindi_sentence_no_punctuations'], axis=1)

In [ ]:
cleaned.head()

In [ ]:
cleaned.tail()

#### Adding Start And End Token

In [ ]:
def Add_Token(sentence):
    sentence = re.sub(r'\s+', ' ', sentence)
    return f"<START> {sentence} <END>"

In [ ]:
cleaned['english_cleaned']= cleaned['english_cleaned'].apply(Add_Token)
cleaned['hindi_cleaned']= cleaned['hindi_cleaned'].apply(Add_Token)

In [ ]:
cleaned.head()

#### DICTIONARY FORMATION

In [ ]:
HINDI_VOCAB = set()
ENGLISH_VOCAB = set()

In [ ]:
def tokenize(sentence):
    temporary_set = set()
    
    for word in sentence.strip().split(' '):
        temporary_set.add(word)
    return temporary_set

In [ ]:
def create_vocab(sentences, dictionary):
    i=0
    for sentence in sentences:
        if i%10000==0:
            print(f"{i} done")
        temp_set= tokenize(sentence)
        dictionary=dictionary.union(temp_set)
        i+=1
        
    return dictionary

In [ ]:
ENGLISH_VOCAB = create_vocab(cleaned['english_cleaned'].tolist(),ENGLISH_VOCAB)
print("English done")
HINDI_VOCAB = create_vocab(cleaned['hindi_cleaned'].tolist(),HINDI_VOCAB)

In [ ]:
len(ENGLISH_VOCAB)
len(HINDI_VOCAB)

In [ ]:
ENGLISH_VOCAB.add("<START>")
ENGLISH_VOCAB.add("<END>")
HINDI_VOCAB.add("<START>")
HINDI_VOCAB.add("<END>")

#### Detecting Max Length for input and output sentences

In [ ]:
ENGLISH_MAX_LENGTH=0
HINDI_MAX_LENGTH=0

In [ ]:
def get_max_length_in_words(sentences):
    length_of_sentences = [len(sentence.split(' ')) for sentence in sentences]
    return np.max(np.array(length_of_sentences))

In [ ]:
ENGLISH_MAX_LENGTH = get_max_length_in_words(cleaned['english_cleaned'].tolist())
HINDI_MAX_LENGTH = get_max_length_in_words(cleaned['hindi_cleaned'].tolist())

In [ ]:
ENGLISH_MAX_LENGTH

In [ ]:
HINDI_MAX_LENGTH

In [ ]:
ENGLISH_AVG_LENGTH=0
HINDI_AVG_LENGTH=0

In [ ]:
def get_avg_length_in_words(sentences):
    length_of_sentences = [len(sentence.split(' ')) for sentence in sentences]
    return np.mean(np.array(length_of_sentences))

In [ ]:
ENGLISH_AVG_LENGTH = get_avg_length_in_words(cleaned['english_cleaned'].tolist())
HINDI_AVG_LENGTH = get_avg_length_in_words(cleaned['hindi_cleaned'].tolist())

In [ ]:
ENGLISH_AVG_LENGTH

In [ ]:
HINDI_AVG_LENGTH

In [ ]:
INPUT_LENGTH = 30
OUTPUT_LENGTH = 30

In [ ]:
def limit_number_of_words_in_the_sentence(sentence):
    words = sentence.split(' ')
    words_capped = words[:INPUT_LENGTH-1]
    new_sentence=""
    
    for word in words_capped:
        new_sentence+=f" {word}"
    
    return new_sentence

In [ ]:
cleaned['english_cleaned'] = cleaned['english_cleaned'].apply(limit_number_of_words_in_the_sentence)
cleaned['hindi_cleaned'] = cleaned['hindi_cleaned'].apply(limit_number_of_words_in_the_sentence)

In [ ]:
cleaned.head()

#### Creating dictionary indexes

In [ ]:
ENGLISH_VOCAB_LIST = sorted(list(ENGLISH_VOCAB))
HINDI_VOCAB_LIST = sorted(list(HINDI_VOCAB))

In [ ]:
english_dictionary_index = dict([(word, i+1) for i, word in enumerate(ENGLISH_VOCAB_LIST)])
hindi_dictionary_index = dict([(word, i+1) for i, word in enumerate(HINDI_VOCAB_LIST)])

In [ ]:
reverse_english_dictionary_index = dict((i, word) for word, i in english_dictionary_index.items())
reverse_hindi_dictionary_index = dict((i, word) for word, i in hindi_dictionary_index.items())

#### Encoding the sentences

In [ ]:
def encode(sentence, index_dictionary):
    
    sentence_tokens = []
    
    for index in range(len(sentence.split(' '))):
        sentence_tokens.append(index_dictionary[sentence.split(' ')[index]])
        
    return sentence_tokens

In [ ]:
cleaned['index_list_english'] = cleaned['english_cleaned'].apply(encode,index_dictionary=english_dictionary_index)

In [ ]:
cleaned['index_list_hindi'] = cleaned['hindi_cleaned'].apply(encode,index_dictionary=hindi_dictionary_index)

#### Adding 0 padding to have proper length

In [ ]:
def pad(index_list):
    pad_seq_size = INPUT_LENGTH - len(index_list)
    zero_arr = np.zeros(pad_seq_size)
    padded_arr = np.concatenate([np.array(index_list),zero_arr])
    
    return padded_arr

In [ ]:
cleaned['index_list_english'] = cleaned['index_list_english'].apply(pad)
cleaned['index_list_hindi'] = cleaned['index_list_hindi'].apply(pad)

In [ ]:
cleaned.head()

#### Creating Test Train Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(cleaned['index_list_english'], cleaned['index_list_hindi'], test_size = 0.3,random_state=42)

#### Data Storing

In [ ]:
import pickle

In [ ]:
with open("/Volumes/T7/ML/Data/Transformers/utilities/X_train.pkl", 'wb') as file:
    pickle.dump(x_train,file)

In [ ]:
with open("/Volumes/T7/ML/Data/Transformers/utilities/Y_train.pkl", 'wb') as file:
    pickle.dump(y_train,file)

In [ ]:
with open("/Volumes/T7/ML/Data/Transformers/utilities/X_test.pkl", 'wb') as file:
    pickle.dump(x_test,file)

In [ ]:
with open("/Volumes/T7/ML/Data/Transformers/utilities/Y_test.pkl", 'wb') as file:
    pickle.dump(y_test,file)

#### Storing dictionaries

In [ ]:
with open("/Volumes/T7/ML/Data/Transformers/utilities/English_Dict.pkl", 'wb') as file:
    pickle.dump(english_dictionary_index,file)

In [ ]:
with open("/Volumes/T7/ML/Data/Transformers/utilities/Hindi_Dict.pkl", 'wb') as file:
    pickle.dump(hindi_dictionary_index,file)

In [ ]:
with open("/Volumes/T7/ML/Data/Transformers/utilities/Reverse_English_Dict.pkl", 'wb') as file:
    pickle.dump(reverse_english_dictionary_index,file)

In [ ]:
with open("/Volumes/T7/ML/Data/Transformers/utilities/Reverse_Hindi_Dict.pkl", 'wb') as file:
    pickle.dump(reverse_hindi_dictionary_index,file)